## NYT COVID-19 Analysis
This notebook processes and performs quick analysis from the [New York Times COVID-19 dataset](https://github.com/nytimes/covid-19-data).  The data is updated in the `/databricks-datasets/COVID/covid-19-data/` location regularly so you can access the data directly.

In [0]:
#!pip install altair
#!pip install vega_datasets

In [0]:
# Standard Libraries
import io

# External Libraries
import requests
import numpy as np
import pandas as pd
import altair as alt
from vega_datasets import data

# # topographical
# topo_usa = 'https://vega.github.io/vega-datasets/data/us-10m.json'
# topo_wa = 'https://raw.githubusercontent.com/deldersveld/topojson/master/countries/us-states/WA-53-washington-counties.json'
# topo_king = 'https://raw.githubusercontent.com/johan/world.geo.json/master/countries/USA/WA/King.geo.json'

In [0]:
# Create mapping of county FIPS to centroid long_ and lat
map_fips = spark.read.option("header", True).option("inferSchema", True).csv("dbfs:/FileStore/shared_uploads/waleakinfaderin@gwu.edu/countyfips_lat_long.csv")
map_fips = (map_fips
              .withColumnRenamed("STATE", "state")
              .withColumnRenamed("COUNTYNAME", "county")
              .withColumnRenamed("LAT", "lat")
              .withColumnRenamed("LON", "long_"))
map_fips.createOrReplaceTempView("map_fips")

In [0]:
map_fips_dedup = spark.sql("""select fips, min(state) as state, min(county) as county, min(long_) as long_, min(lat) as lat from map_fips group by fips""")
map_fips_dedup.createOrReplaceTempView("map_fips_dedup")

### Download 2019 Population Estimates

In [0]:
map_popest_county = spark.read.option("header", True).option("inferSchema", True).csv("dbfs:/FileStore/shared_uploads/waleakinfaderin@gwu.edu/co_est2019_alldata.csv")
map_popest_county.createOrReplaceTempView("map_popest_county")
fips_popest_county = spark.sql("select State * 1000 + substring(cast(1000 + County as string), 2, 3) as fips, STNAME, CTYNAME, census2010pop, POPESTIMATE2019 from map_popest_county")
fips_popest_county.createOrReplaceTempView("fips_popest_county")

## Specify `nyt_daily` table
* Source: `/databricks-datasets/COVID/covid-19-data/`
* Contains the COVID-19 daily reports

In [0]:
nyt_daily = spark.read.option("inferSchema", True).option("header", True).csv("/databricks-datasets/COVID/covid-19-data/us-counties.csv")
nyt_daily.createOrReplaceTempView("nyt_daily")
display(nyt_daily)

date county state fips cases deaths 2020-01-21 Snohomish Washington 53061 1 0 2020-01-22 Snohomish Washington 53061 1 0 2020-01-23 Snohomish Washington 53061 1 0 2020-01-24 Cook Illinois 17031 1 0 2020-01-24 Snohomish Washington 53061 1 0 2020-01-25 Orange California 6059 1 0 2020-01-25 Cook Illinois 17031 1 0 2020-01-25 Snohomish Washington 53061 1 0 2020-01-26 Maricopa Arizona 4013 1 0 2020-01-26 Los Angeles California 6037 1 0 2020-01-26 Orange California 6059 1 0 2020-01-26 Cook Illinois 17031 1 0 2020-01-26 Snohomish Washington 53061 1 0 2020-01-27 Maricopa Arizona 4013 1 0 2020-01-27 Los Angeles California 6037 1 0 2020-01-27 Orange California 6059 1 0 2020-01-27 Cook Illinois 17031 1 0 2020-01-27 Snohomish Washington 53061 1 0 2020-01-28 Maricopa Arizona 4013 1 0 2020-01-28 Los Angeles California 6037 1 0 2020-01-28 Orange California 6059 1 0 2020-01-28 Cook Illinois 17031 1 0 2020-01-28 Snohomish Washington 53061 1 0 2020-01-29 Maricopa Arizona 4013 1 0 2020-01-29 Los Angeles California 6037 1 0 2020-01-29 Orange California 6059 1 0 2020-01-29 Cook Illinois 17031 1 0 2020-01-29 Snohomish Washington 53061 1 0 2020-01-30 Maricopa Arizona 4013 1 0 2020-01-30 Los Angeles California 6037 1 0 2020-01-30 Orange California 6059 1 0 2020-01-30 Cook Illinois 17031 2 0 2020-01-30 Snohomish Washington 53061 1 0 2020-01-31 Maricopa Arizona 4013 1 0 2020-01-31 Los Angeles California 6037 1 0 2020-01-31 Orange California 6059 1 0 2020-01-31 Santa Clara California 6085 1 0 2020-01-31 Cook Illinois 17031 2 0 2020-01-31 Snohomish Washington 53061 1 0 2020-02-01 Maricopa Arizona 4013 1 0 2020-02-01 Los Angeles California 6037 1 0 2020-02-01 Orange California 6059 1 0 2020-02-01 Santa Clara California 6085 1 0 2020-02-01 Cook Illinois 17031 2 0 2020-02-01 Suffolk Massachusetts 25025 1 0 2020-02-01 Snohomish Washington 53061 1 0 2020-02-02 Maricopa Arizona 4013 1 0 2020-02-02 Los Angeles California 6037 1 0 2020-02-02 Orange California 6059 1 0 2020-02-02 San Francisco California 6075 2 0 2020-02-02 Santa Clara California 6085 2 0 2020-02-02 Cook Illinois 17031 2 0 2020-02-02 Suffolk Massachusetts 25025 1 0 2020-02-02 Snohomish Washington 53061 1 0 2020-02-03 Maricopa Arizona 4013 1 0 2020-02-03 Los Angeles California 6037 1 0 2020-02-03 Orange California 6059 1 0 2020-02-03 San Francisco California 6075 2 0 2020-02-03 Santa Clara California 6085 2 0 2020-02-03 Cook Illinois 17031 2 0 2020-02-03 Suffolk Massachusetts 25025 1 0 2020-02-03 Snohomish Washington 53061 1 0 2020-02-04 Maricopa Arizona 4013 1 0 2020-02-04 Los Angeles California 6037 1 0 2020-02-04 Orange California 6059 1 0 2020-02-04 San Francisco California 6075 2 0 2020-02-04 Santa Clara California 6085 2 0 2020-02-04 Cook Illinois 17031 2 0 2020-02-04 Suffolk Massachusetts 25025 1 0 2020-02-04 Snohomish Washington 53061 1 0 2020-02-05 Maricopa Arizona 4013 1 0 2020-02-05 Los Angeles California 6037 1 0 2020-02-05 Orange California 6059 1 0 2020-02-05 San Francisco California 6075 2 0 2020-02-05 Santa Clara California 6085 2 0 2020-02-05 Cook Illinois 17031 2 0 2020-02-05 Suffolk Massachusetts 25025 1 0 2020-02-05 Snohomish Washington 53061 1 0 2020-02-05 Dane Wisconsin 55025 1 0 2020-02-06 Maricopa Arizona 4013 1 0 2020-02-06 Los Angeles California 6037 1 0 2020-02-06 Orange California 6059 1 0 2020-02-06 San Francisco California 6075 2 0 2020-02-06 Santa Clara California 6085 2 0 2020-02-06 Cook Illinois 17031 2 0 2020-02-06 Suffolk Massachusetts 25025 1 0 2020-02-06 Snohomish Washington 53061 1 0 2020-02-06 Dane Wisconsin 55025 1 0 2020-02-07 Maricopa Arizona 4013 1 0 2020-02-07 Los Angeles California 6037 1 0 2020-02-07 Orange California 6059 1 0 2020-02-07 San Francisco California 6075 2 0 2020-02-07 Santa Clara California 6085 2 0 2020-02-07 Cook Illinois 17031 2 0 2020-02-07 Suffolk Massachusetts 25025 1 0 2020-02-07 Snohomish Washington 53061 1 0 2020-02-07 Dane Wisconsin 55025 1 0 2020-02-08 Maricopa Arizona 4013 1 0 2020-02-08 Los Angeles California 6037 1 0 2020-02-

# COVID-19 Cases and Deaths for Specific Counties
Focusing on two-week window around when educational facilites were closed
* Top 10 Washington State counties (3/13/2020)
* Top 10 NY State counties (3/18/2020)
The queries are using the US Census Population Estimates for 2019

In [0]:
# WA State 2 week window
wa_state_window = spark.sql("""
SELECT date, 100 + datediff(date, '2020-03-06T00:00:00.000+0000') as day_num, county, fips, cases, deaths, 100000.*cases/population_estimate AS cases_per_100Kpop, 100000.*deaths/population_estimate AS deaths_per_100Kpop
  from (
SELECT CAST(f.date AS date) AS date, f.county, f.fips, SUM(f.cases) AS cases, SUM(f.deaths) AS deaths, MAX(p.POPESTIMATE2019) AS population_estimate 
  FROM nyt_daily f 
    JOIN fips_popest_county p
      ON p.fips = f.fips
 WHERE f.state = 'Washington' 
   AND date BETWEEN '2020-03-06T00:00:00.000+0000' AND '2020-03-20T00:00:00.000+0000'
 GROUP BY f.date, f.county, f.fips
) a""")
wa_state_window.createOrReplaceTempView("wa_state_window")

# NY State 2 week window
ny_state_window = spark.sql("""
SELECT date, 100 + datediff(date, '2020-03-11T00:00:00.000+0000') as day_num, county, fips, cases, deaths, 100000.*cases/population_estimate AS cases_per_100Kpop, 100000.*deaths/population_estimate AS deaths_per_100Kpop
  FROM (
SELECT CAST(f.date AS date) AS date, f.county, p.fips, SUM(f.cases) as cases, SUM(f.deaths) as deaths, MAX(p.POPESTIMATE2019) AS population_estimate  
  FROM nyt_daily f 
    JOIN fips_popest_county p
      ON p.fips = coalesce(f.fips, 36061)
 WHERE f.state = 'New York' 
   AND date BETWEEN '2020-03-11T00:00:00.000+0000' AND '2020-03-25T00:00:00.000+0000'
 GROUP BY f.date, f.county, p.fips
) a""")
ny_state_window.createOrReplaceTempView("ny_state_window")

# NY State 2 week window (-1 week)
ny_state_window_m1 = spark.sql("""
SELECT date, 100 + datediff(date, '2020-03-06T00:00:00.000+0000') as day_num, county, fips, cases, deaths, 100000.*cases/population_estimate AS cases_per_100Kpop, 100000.*deaths/population_estimate AS deaths_per_100Kpop
  FROM (
SELECT CAST(f.date AS date) AS date, f.county, p.fips, SUM(f.cases) as cases, SUM(f.deaths) as deaths, MAX(p.POPESTIMATE2019) AS population_estimate  
  FROM nyt_daily f 
    JOIN fips_popest_county p
      ON p.fips = coalesce(f.fips, 36061)
 WHERE f.state = 'New York' 
   AND date BETWEEN '2020-03-06T00:00:00.000+0000' AND '2020-03-20T00:00:00.000+0000'
 GROUP BY f.date, f.county, p.fips
) a""")
ny_state_window_m1.createOrReplaceTempView("ny_state_window_m1")

## COVID-19 Cases for WA and NY Counties

In [0]:
%sql
SELECT f.date, f.county, f.cases 
  FROM wa_state_window f
  JOIN (
      SELECT county, sum(cases) as cases FROM wa_state_window GROUP BY county ORDER BY cases DESC LIMIT 10
    ) x ON x.county = f.county

date county cases 2020-03-06 Pierce 1 2020-03-11 Snohomish 69 2020-03-20 Spokane 11 2020-03-13 Yakima 2 2020-03-15 Skagit 4 2020-03-12 Yakima 2 2020-03-15 Kitsap 5 2020-03-12 Whatcom 1 2020-03-18 King 563 2020-03-18 Kitsap 9 2020-03-15 Whatcom 3 2020-03-16 Thurston 4 2020-03-12 Pierce 17 2020-03-17 Kitsap 7 2020-03-08 Spokane 4 2020-03-11 King 235 2020-03-07 Pierce 3 2020-03-14 Thurston 3 2020-03-16 Kitsap 7 2020-03-09 King 117 2020-03-12 Spokane 4 2020-03-13 Kitsap 3 2020-03-06 Snohomish 18 2020-03-18 Pierce 56 2020-03-19 Kitsap 12 2020-03-20 Kitsap 12 2020-03-10 Whatcom 1 2020-03-18 Skagit 14 2020-03-17 Yakima 5 2020-03-16 Spokane 7 2020-03-14 Snohomish 154 2020-03-17 Thurston 5 2020-03-17 King 518 2020-03-09 Kitsap 1 2020-03-18 Whatcom 7 2020-03-11 Skagit 1 2020-03-19 Island 17 2020-03-18 Thurston 6 2020-03-11 Kitsap 2 2020-03-12 Skagit 2 2020-03-16 Skagit 7 2020-03-13 Pierce 19 2020-03-19 Spokane 9 2020-03-20 Whatcom 11 2020-03-06 King 59 2020-03-10 Kitsap 2 2020-03-17 Spokane 7 2020-03-08 Kitsap 1 2020-03-08 Snohomish 36 2020-03-20 Island 19 2020-03-12 Kitsap 2 2020-03-17 Snohomish 265 2020-03-10 Snohomish 61 2020-03-13 Whatcom 1 2020-03-16 Pierce 44 2020-03-18 Yakima 9 2020-03-13 King 329 2020-03-08 Pierce 4 2020-03-12 Thurston 1 2020-03-13 Skagit 3 2020-03-14 King 389 2020-03-19 Thurston 6 2020-03-20 King 794 2020-03-10 King 191 2020-03-16 Island 7 2020-03-12 Snohomish 107 2020-03-20 Thurston 8 2020-03-17 Whatcom 6 2020-03-19 Whatcom 7 2020-03-16 Whatcom 3 2020-03-16 Yakima 4 2020-03-16 Snohomish 199 2020-03-09 Snohomish 45 2020-03-11 Whatcom 1 2020-03-12 Island 3 2020-03-11 Thurston 1 2020-03-14 Spokane 7 2020-03-15 Spokane 7 2020-03-14 Whatcom 2 2020-03-17 Skagit 9 2020-03-11 Pierce 17 2020-03-19 Snohomish 347 2020-03-15 Snohomish 175 2020-03-19 Skagit 18 2020-03-20 Yakima 10 2020-03-13 Island 3 2020-03-17 Island 14 2020-03-13 Thurston 3 2020-03-15 Thurston 4 2020-03-16 King 489 2020-03-19 King 693 2020-03-11 Island 1 2020-03-07 Snohomish 26 2020-03-09 Spokane 4 2020-03-14 Skagit 3 2020-03-18 Island 16 2020-03-12 King 271 2020-03-19 Pierce 75 2020-03-15 Pierce 29 2020-03-18 Spokane 7 2020-03-15 King 421 2020-03-17 Pierce 44 2020-03-06 Spokane 4 2020-03-20 Skagit 26 2020-03-15 Yakima 4 2020-03-19 Yakima 9 2020-03-10 Skagit 1 2020-03-14 Island 6 2020-03-10 Spokane 4 2020-03-13 Spokane 4 2020-03-14 Pierce 26 2020-03-07 King 72 2020-03-14 Kitsap 3 2020-03-14 Yakima 3 2020-03-15 Island 6 2020-03-20 Snohomish 384 2020-03-20 Pierce 83 2020-03-08 King 91 2020-03-13 Snohomish 132 2020-03-07 Spokane 4 2020-03-09 Pierce 7 2020-03-10 Pierce 14 2020-03-11 Spokane 4 2020-03-18 Snohomish 309

In [0]:
%sql
SELECT f.date, f.county, f.cases 
  FROM ny_state_window f
  JOIN (
      SELECT county, sum(cases) as cases FROM ny_state_window GROUP BY county ORDER BY cases DESC LIMIT 10
    ) x ON x.county = f.county

date county cases 2020-03-25 Orange 638 2020-03-22 Suffolk 1034 2020-03-15 Nassau 98 2020-03-11 New York City 52 2020-03-19 Nassau 382 2020-03-21 Albany 88 2020-03-23 Orange 389 2020-03-14 Orange 6 2020-03-16 Rockland 16 2020-03-14 Albany 5 2020-03-14 Westchester 178 2020-03-18 New York City 1344 2020-03-23 Westchester 2894 2020-03-23 Rockland 592 2020-03-18 Monroe 14 2020-03-12 Suffolk 21 2020-03-21 New York City 6226 2020-03-21 Westchester 1387 2020-03-16 Suffolk 63 2020-03-20 Dutchess 36 2020-03-23 Dutchess 100 2020-03-15 Albany 11 2020-03-20 Westchester 1091 2020-03-25 Dutchess 153 2020-03-16 New York City 464 2020-03-17 Nassau 131 2020-03-21 Dutchess 49 2020-03-17 Westchester 379 2020-03-25 Suffolk 2260 2020-03-21 Nassau 1234 2020-03-24 Suffolk 1880 2020-03-25 Albany 152 2020-03-15 Orange 6 2020-03-12 Nassau 41 2020-03-16 Erie 6 2020-03-17 Erie 7 2020-03-24 Dutchess 124 2020-03-23 Monroe 76 2020-03-12 Orange 1 2020-03-17 Monroe 11 2020-03-22 New York City 9065 2020-03-24 Westchester 3891 2020-03-12 Albany 2 2020-03-20 Monroe 32 2020-03-16 Albany 12 2020-03-18 Suffolk 116 2020-03-19 Albany 43 2020-03-20 New York City 4419 2020-03-24 Monroe 96 2020-03-15 Suffolk 47 2020-03-19 New York City 2477 2020-03-19 Dutchess 31 2020-03-23 Albany 127 2020-03-25 Rockland 1069 2020-03-25 Westchester 4691 2020-03-24 Rockland 671 2020-03-25 New York City 20061 2020-03-18 Dutchess 20 2020-03-15 Dutchess 7 2020-03-18 Rockland 30 2020-03-13 Albany 2 2020-03-19 Rockland 53 2020-03-13 Orange 3 2020-03-19 Monroe 28 2020-03-24 Erie 107 2020-03-19 Westchester 797 2020-03-17 Albany 23 2020-03-19 Erie 28 2020-03-21 Suffolk 662 2020-03-21 Erie 38 2020-03-20 Nassau 754 2020-03-15 Westchester 195 2020-03-17 Suffolk 84 2020-03-22 Erie 54 2020-03-13 New York City 155 2020-03-23 New York City 12329 2020-03-16 Nassau 109 2020-03-12 Monroe 1 2020-03-18 Westchester 537 2020-03-18 Orange 32 2020-03-13 Rockland 9 2020-03-11 Nassau 28 2020-03-16 Dutchess 10 2020-03-12 Westchester 147 2020-03-21 Orange 163 2020-03-19 Suffolk 178 2020-03-16 Monroe 10 2020-03-15 Erie 3 2020-03-23 Suffolk 1458 2020-03-11 Rockland 6 2020-03-22 Monroe 57 2020-03-23 Erie 87 2020-03-24 Orange 498 2020-03-19 Orange 51 2020-03-24 New York City 14943 2020-03-11 Westchester 120 2020-03-20 Albany 61 2020-03-15 Rockland 13 2020-03-16 Westchester 219 2020-03-11 Monroe 1 2020-03-25 Monroe 118 2020-03-17 Rockland 22 2020-03-18 Erie 7 2020-03-13 Westchester 157 2020-03-12 Rockland 7 2020-03-17 New York City 646 2020-03-20 Suffolk 371 2020-03-14 Monroe 2 2020-03-12 Dutchess 1 2020-03-21 Monroe 42 2020-03-18 Nassau 183 2020-03-21 Rockland 262 2020-03-22 Westchester 1873 2020-03-24 Nassau 2869 2020-03-14 New York City 269 2020-03-14 Rockland 12 2020-03-12 New York City 96 2020-03-20 Rockland 101 2020-03-22 Orange 247 2020-03-24 Albany 146 2020-03-13 Suffolk 29 2020-03-14 Nassau 79 2020-03-13 Nassau 51 2020-03-15 Monroe 2 2020-03-25 Nassau 3285 2020-03-22 Nassau 1900 2020-03-16 Orange 11 2020-03-15 New York City 330 2020-03-23 Nassau 2442 2020-03-14 Suffolk 41 2020-03-17 Dutchess 16 2020-03-22 Albany 123 2020-03-22 Rockland 455 2020-03-13 Dutchess 3 2020-03-18 Albany 36 2020-03-22 Dutchess 82 2020-03-11 Suffolk 6 2020-03-13 Monroe 1 2020-03-20 Erie 31 2020-03-20 Orange 84 2020-03-25 Erie 122 2020-03-14 Dutchess 4 2020-03-17 Orange 15

In [0]:
%sql
SELECT f.date, f.county, f.cases 
  FROM ny_state_window_m1 f
  JOIN (
      SELECT county, sum(cases) as cases FROM ny_state_window_m1 GROUP BY county ORDER BY cases DESC LIMIT 10
    ) x ON x.county = f.county

date county cases 2020-03-09 New York City 20 2020-03-15 Nassau 98 2020-03-11 New York City 52 2020-03-08 New York City 14 2020-03-19 Nassau 382 2020-03-14 Orange 6 2020-03-16 Rockland 16 2020-03-14 Albany 5 2020-03-14 Westchester 178 2020-03-09 Nassau 17 2020-03-15 Saratoga 3 2020-03-07 Saratoga 2 2020-03-18 New York City 1344 2020-03-18 Monroe 14 2020-03-10 Suffolk 1 2020-03-12 Suffolk 21 2020-03-16 Suffolk 63 2020-03-20 Dutchess 36 2020-03-15 Albany 11 2020-03-20 Westchester 1091 2020-03-16 New York City 464 2020-03-17 Nassau 131 2020-03-07 Rockland 2 2020-03-17 Westchester 379 2020-03-07 Westchester 69 2020-03-10 Nassau 19 2020-03-11 Saratoga 2 2020-03-15 Orange 6 2020-03-09 Suffolk 1 2020-03-12 Nassau 41 2020-03-06 Nassau 4 2020-03-06 New York City 5 2020-03-16 Saratoga 5 2020-03-12 Orange 1 2020-03-17 Monroe 11 2020-03-12 Albany 2 2020-03-20 Monroe 32 2020-03-16 Albany 12 2020-03-08 Nassau 5 2020-03-18 Suffolk 116 2020-03-18 Saratoga 14 2020-03-19 Albany 43 2020-03-08 Saratoga 2 2020-03-19 Saratoga 14 2020-03-20 New York City 4419 2020-03-20 Saratoga 24 2020-03-08 Westchester 81 2020-03-13 Saratoga 3 2020-03-08 Rockland 2 2020-03-15 Suffolk 47 2020-03-19 New York City 2477 2020-03-19 Dutchess 31 2020-03-18 Dutchess 20 2020-03-12 Saratoga 3 2020-03-15 Dutchess 7 2020-03-18 Rockland 30 2020-03-13 Albany 2 2020-03-19 Rockland 53 2020-03-13 Orange 3 2020-03-19 Monroe 28 2020-03-19 Westchester 797 2020-03-17 Albany 23 2020-03-20 Nassau 754 2020-03-15 Westchester 195 2020-03-17 Suffolk 84 2020-03-13 New York City 155 2020-03-16 Nassau 109 2020-03-12 Monroe 1 2020-03-18 Westchester 537 2020-03-18 Orange 32 2020-03-13 Rockland 9 2020-03-11 Nassau 28 2020-03-16 Dutchess 10 2020-03-12 Westchester 147 2020-03-14 Saratoga 3 2020-03-10 New York City 37 2020-03-19 Suffolk 178 2020-03-16 Monroe 10 2020-03-10 Westchester 107 2020-03-11 Rockland 6 2020-03-09 Saratoga 2 2020-03-19 Orange 51 2020-03-11 Westchester 120 2020-03-20 Albany 61 2020-03-09 Rockland 4 2020-03-15 Rockland 13 2020-03-16 Westchester 219 2020-03-09 Westchester 97 2020-03-11 Monroe 1 2020-03-17 Saratoga 9 2020-03-17 Rockland 22 2020-03-13 Westchester 157 2020-03-12 Rockland 7 2020-03-17 New York City 646 2020-03-20 Suffolk 371 2020-03-14 Monroe 2 2020-03-12 Dutchess 1 2020-03-18 Nassau 183 2020-03-06 Rockland 2 2020-03-14 New York City 269 2020-03-14 Rockland 12 2020-03-12 New York City 96 2020-03-20 Rockland 101 2020-03-13 Suffolk 29 2020-03-14 Nassau 79 2020-03-07 New York City 12 2020-03-10 Saratoga 2 2020-03-10 Rockland 6 2020-03-13 Nassau 51 2020-03-15 Monroe 2 2020-03-16 Orange 11 2020-03-15 New York City 330 2020-03-14 Suffolk 41 2020-03-17 Dutchess 16 2020-03-13 Dutchess 3 2020-03-18 Albany 36 2020-03-07 Nassau 4 2020-03-11 Suffolk 6 2020-03-13 Monroe 1 2020-03-06 Westchester 33 2020-03-08 Suffolk 1 2020-03-20 Orange 84 2020-03-14 Dutchess 4 2020-03-17 Orange 15

## COVID-19 Cases per 100K people for WA and NY Counties
The concern with the above graphs is while it provides you the total number of cases, it's hard to compare WA state and NY state due to density.  While not perfect, a better way to look at these numbers would be to review this data as a proportion of population estimates.

Let's look at these values by a percentage of the population; the numbers used are the 2019 US Census estimates of county populations.

*Note, reviewing the top 10 counties by case (vs. % of cases)*

In [0]:
%sql
SELECT f.date, f.county, f.cases_per_100Kpop, f.cases
  FROM wa_state_window f
  JOIN (
      SELECT county, sum(cases) as cases FROM wa_state_window GROUP BY county ORDER BY cases DESC LIMIT 10  
    ) x ON x.county = f.county

date county cases_per_100Kpop cases 2020-03-06 Pierce 0.11049967955 1 2020-03-11 Snohomish 8.39331308396 69 2020-03-20 Spokane 2.10406313720 11 2020-03-13 Yakima 0.79721612130 2 2020-03-15 Skagit 3.09585542355 4 2020-03-12 Yakima 0.79721612130 2 2020-03-15 Kitsap 1.84180378896 5 2020-03-12 Whatcom 0.43621072468 1 2020-03-18 King 24.99132184117 563 2020-03-18 Kitsap 3.31524682013 9 2020-03-15 Whatcom 1.30863217403 3 2020-03-16 Thurston 1.37676570201 4 2020-03-12 Pierce 1.87849455237 17 2020-03-17 Kitsap 2.57852530454 7 2020-03-08 Spokane 0.76511386807 4 2020-03-11 King 10.43154641683 235 2020-03-07 Pierce 0.33149903865 3 2020-03-14 Thurston 1.03257427651 3 2020-03-16 Kitsap 2.57852530454 7 2020-03-09 King 5.19357842880 117 2020-03-12 Spokane 0.76511386807 4 2020-03-13 Kitsap 1.10508227338 3 2020-03-06 Snohomish 2.18955993495 18 2020-03-18 Pierce 6.18798205485 56 2020-03-19 Kitsap 4.42032909350 12 2020-03-20 Kitsap 4.42032909350 12 2020-03-10 Whatcom 0.43621072468 1 2020-03-18 Skagit 10.83549398243 14 2020-03-17 Yakima 1.99304030326 5 2020-03-16 Spokane 1.33894926912 7 2020-03-14 Snohomish 18.73290166565 154 2020-03-17 Thurston 1.72095712752 5 2020-03-17 King 22.99379167625 518 2020-03-09 Kitsap 0.36836075779 1 2020-03-18 Whatcom 3.05347507274 7 2020-03-11 Skagit 0.77396385589 1 2020-03-19 Island 19.96687847218 17 2020-03-18 Thurston 2.06514855302 6 2020-03-11 Kitsap 0.73672151558 2 2020-03-12 Skagit 1.54792771178 2 2020-03-16 Skagit 5.41774699122 7 2020-03-13 Pierce 2.09949391147 19 2020-03-19 Spokane 1.72150620316 9 2020-03-20 Whatcom 4.79831797145 11 2020-03-06 King 2.61898399401 59 2020-03-10 Kitsap 0.73672151558 2 2020-03-17 Spokane 1.33894926912 7 2020-03-08 Kitsap 0.36836075779 1 2020-03-08 Snohomish 4.37911986989 36 2020-03-20 Island 22.31592299832 19 2020-03-12 Kitsap 0.73672151558 2 2020-03-17 Snohomish 32.23518793115 265 2020-03-10 Snohomish 7.42017533509 61 2020-03-13 Whatcom 0.43621072468 1 2020-03-16 Pierce 4.86198590024 44 2020-03-18 Yakima 3.58747254587 9 2020-03-13 King 14.60416498356 329 2020-03-08 Pierce 0.44199871820 4 2020-03-12 Thurston 0.34419142550 1 2020-03-13 Skagit 2.32189156766 3 2020-03-14 King 17.26753853680 389 2020-03-19 Thurston 2.06514855302 6 2020-03-20 King 35.24531002112 794 2020-03-10 King 8.47840581113 191 2020-03-16 Island 8.22165584149 7 2020-03-12 Snohomish 13.01571739107 107 2020-03-20 Thurston 2.75353140403 8 2020-03-17 Whatcom 2.61726434806 6 2020-03-19 Whatcom 3.05347507274 7 2020-03-16 Whatcom 1.30863217403 3 2020-03-16 Yakima 1.59443224261 4 2020-03-16 Snohomish 24.20680150301 199 2020-03-09 Snohomish 5.47389983736 45 2020-03-11 Whatcom 0.43621072468 1 2020-03-12 Island 3.52356678921 3 2020-03-11 Thurston 0.34419142550 1 2020-03-14 Spokane 1.33894926912 7 2020-03-15 Spokane 1.33894926912 7 2020-03-14 Whatcom 0.87242144935 2 2020-03-17 Skagit 6.96567470299 9 2020-03-11 Pierce 1.87849455237 17 2020-03-19 Snohomish 42.20984985701 347 2020-03-15 Snohomish 21.28738825642 175 2020-03-19 Skagit 13.93134940598 18 2020-03-20 Yakima 3.98608060652 10 2020-03-13 Island 3.52356678921 3 2020-03-17 Island 16.44331168297 14 2020-03-13 Thurston 1.03257427651 3 2020-03-15 Thurston 1.37676570201 4 2020-03-16 King 21.70649445885 489 2020-03-19 King 30.76196453984 693 2020-03-11 Island 1.17452226307 1 2020-03-07 Snohomish 3.16269768381 26 2020-03-09 Spokane 0.76511386807 4 2020-03-14 Skagit 2.32189156766 3 2020-03-18 Island 18.79235620911 16 2020-03-12 King 12.02957054877 271 2020-03-19 Pierce 8.28747596632 75 2020-03-15 Pierce 3.20449070698 29 2020-03-18 Spokane 1.33894926912 7 2020-03-15 King 18.68800443185 421 2020-03-17 Pierce 4.86198590024 44 2020-03-06 Spokane 0.76511386807 4 2020-03-20 Skagit 20.12306025309 26 2020-03-15 Yakima 1.59443224261 4 2020-03-19 Yakima 3.58747254587 9 2020-03-10 Skagit 0.77396385589 1 2020-03-14 Island 7.04713357842 6 2020-03-10 Spokane 0.76511386807 4 2020-03-13 Spokane 0.76511386807 4 2020-03-14 Pierce 2.87299166832 26 2020-03-07 King 3.19604826388 72 2020-03-14 Kitsa

In [0]:
%sql
SELECT f.date, f.county, f.cases_per_100Kpop 
  FROM ny_state_window f
  JOIN (
      SELECT county, sum(cases) as cases FROM ny_state_window GROUP BY county ORDER BY cases  DESC LIMIT 10
    ) x ON x.county = f.county

date county cases_per_100Kpop 2020-03-25 Orange 165.74011534265 2020-03-22 Suffolk 70.02568737255 2020-03-15 Nassau 7.22221730915 2020-03-11 New York City 3.19271863676 2020-03-19 Nassau 28.15190828668 2020-03-21 Albany 28.80467159401 2020-03-23 Orange 101.05470982491 2020-03-14 Orange 1.55868447031 2020-03-16 Rockland 4.91115415192 2020-03-14 Albany 1.63662906784 2020-03-14 Westchester 18.39781872154 2020-03-18 New York City 82.51949707314 2020-03-23 Westchester 299.11959202320 2020-03-23 Rockland 181.71270362106 2020-03-18 Monroe 1.88737748898 2020-03-12 Suffolk 1.42218514006 2020-03-21 New York City 382.26665831648 2020-03-21 Westchester 143.35828408299 2020-03-16 Suffolk 4.26655542018 2020-03-20 Dutchess 12.23582513646 2020-03-23 Dutchess 33.98840315684 2020-03-15 Albany 3.60058394925 2020-03-20 Westchester 112.76415856853 2020-03-25 Dutchess 52.00225682997 2020-03-16 New York City 28.48887398954 2020-03-17 Nassau 9.65418844386 2020-03-21 Dutchess 16.65431754685 2020-03-17 Westchester 39.17288368238 2020-03-25 Suffolk 153.05421031138 2020-03-21 Nassau 90.94098121929 2020-03-24 Suffolk 127.31943158646 2020-03-25 Albany 49.75352366238 2020-03-15 Orange 1.55868447031 2020-03-12 Nassau 3.02153989464 2020-03-16 Erie 0.65309534539 2020-03-17 Erie 0.76194456962 2020-03-24 Dutchess 42.14561991449 2020-03-23 Monroe 10.24576351160 2020-03-12 Orange 0.25978074505 2020-03-17 Monroe 1.48293945563 2020-03-22 New York City 556.57681619642 2020-03-24 Westchester 402.16804857024 2020-03-12 Albany 0.65465162714 2020-03-20 Monroe 4.31400568910 2020-03-16 Albany 3.92790976282 2020-03-18 Suffolk 7.85587982129 2020-03-19 Albany 14.07500998344 2020-03-20 New York City 271.31968568913 2020-03-24 Monroe 12.94201706729 2020-03-15 Suffolk 3.18298578966 2020-03-19 New York City 152.08392429327 2020-03-19 Dutchess 10.53640497862 2020-03-23 Albany 41.57037832318 2020-03-25 Rockland 328.12648677518 2020-03-25 Westchester 484.85487428502 2020-03-24 Rockland 205.96152724616 2020-03-25 New York City 1231.71401100014 2020-03-18 Dutchess 6.79768063137 2020-03-15 Dutchess 2.37918822098 2020-03-18 Rockland 9.20841403485 2020-03-13 Albany 0.65465162714 2020-03-19 Rockland 16.26819812824 2020-03-13 Orange 0.77934223515 2020-03-19 Monroe 3.77475497796 2020-03-24 Erie 11.64686699278 2020-03-19 Westchester 82.37675011835 2020-03-17 Albany 7.52849371207 2020-03-19 Erie 3.04777827848 2020-03-21 Suffolk 44.83269346289 2020-03-21 Erie 4.13627052080 2020-03-20 Nassau 55.56685562345 2020-03-15 Westchester 20.15491376798 2020-03-17 Suffolk 5.68874056025 2020-03-22 Erie 5.87785810851 2020-03-13 New York City 9.51675747495 2020-03-23 New York City 756.98130908832 2020-03-16 Nassau 8.03287435405 2020-03-12 Monroe 0.13481267778 2020-03-18 Westchester 55.50353176104 2020-03-18 Orange 8.31298384164 2020-03-13 Rockland 2.76252421046 2020-03-11 Nassau 2.06349065976 2020-03-16 Dutchess 3.39884031568 2020-03-12 Westchester 15.19370422509 2020-03-21 Orange 42.34426144334 2020-03-19 Suffolk 12.05471213957 2020-03-16 Monroe 1.34812677784 2020-03-15 Erie 0.32654767269 2020-03-23 Suffolk 98.74028258142 2020-03-11 Rockland 1.84168280697 2020-03-22 Monroe 7.68432263370 2020-03-23 Erie 9.46988250815 2020-03-24 Orange 129.37081103549 2020-03-19 Orange 13.24881799761 2020-03-24 New York City 917.47681902074 2020-03-11 Westchester 12.40302385722 2020-03-20 Albany 19.96687462767 2020-03-15 Rockland 3.99031274844 2020-03-16 Westchester 22.63551853942 2020-03-11 Monroe 0.13481267778 2020-03-25 Monroe 15.90789597854 2020-03-17 Rockland 6.75283695889 2020-03-18 Erie 0.76194456962 2020-03-13 Westchester 16.22728954652 2020-03-12 Rockland 2.14862994147 2020-03-17 New York City 39.66338921819 2020-03-20 Suffolk 25.12527080775 2020-03-14 Monroe 0.26962535557 2020-03-12 Dutchess 0.33988403157 2020-03-21 Monroe 5.66213246694 2020-03-18 Nassau 13.48638538341 2020-03-21 Rockland 80.42014923770 2020-03-22 Westchester 193.59053070472 2020-03-24 Nassau 211.43409653009 2020-03-14 New York City 16.51617910169 

In [0]:
%sql
SELECT f.date, f.county, f.cases_per_100Kpop 
  FROM ny_state_window_m1 f
  JOIN (
      SELECT county, sum(cases) as cases FROM ny_state_window_m1 GROUP BY county ORDER BY cases DESC LIMIT 10
    ) x ON x.county = f.county

date county cases_per_100Kpop 2020-03-09 New York City 1.22796870645 2020-03-15 Nassau 7.22221730915 2020-03-11 New York City 3.19271863676 2020-03-08 New York City 0.85957809451 2020-03-19 Nassau 28.15190828668 2020-03-14 Orange 1.55868447031 2020-03-16 Rockland 4.91115415192 2020-03-14 Albany 1.63662906784 2020-03-14 Westchester 18.39781872154 2020-03-09 Nassau 1.25283361485 2020-03-15 Saratoga 1.30512522677 2020-03-07 Saratoga 0.87008348451 2020-03-18 New York City 82.51949707314 2020-03-18 Monroe 1.88737748898 2020-03-10 Suffolk 0.06772310191 2020-03-12 Suffolk 1.42218514006 2020-03-16 Suffolk 4.26655542018 2020-03-20 Dutchess 12.23582513646 2020-03-15 Albany 3.60058394925 2020-03-20 Westchester 112.76415856853 2020-03-16 New York City 28.48887398954 2020-03-17 Nassau 9.65418844386 2020-03-07 Rockland 0.61389426899 2020-03-17 Westchester 39.17288368238 2020-03-07 Westchester 7.13173871790 2020-03-10 Nassau 1.40022580484 2020-03-11 Saratoga 0.87008348451 2020-03-15 Orange 1.55868447031 2020-03-09 Suffolk 0.06772310191 2020-03-12 Nassau 3.02153989464 2020-03-06 Nassau 0.29478437997 2020-03-06 New York City 0.30699217661 2020-03-16 Saratoga 2.17520871128 2020-03-12 Orange 0.25978074505 2020-03-17 Monroe 1.48293945563 2020-03-12 Albany 0.65465162714 2020-03-20 Monroe 4.31400568910 2020-03-16 Albany 3.92790976282 2020-03-08 Nassau 0.36848047496 2020-03-18 Suffolk 7.85587982129 2020-03-18 Saratoga 6.09058439157 2020-03-19 Albany 14.07500998344 2020-03-08 Saratoga 0.87008348451 2020-03-19 Saratoga 6.09058439157 2020-03-20 New York City 271.31968568913 2020-03-20 Saratoga 10.44100181412 2020-03-08 Westchester 8.37204110362 2020-03-13 Saratoga 1.30512522677 2020-03-08 Rockland 0.61389426899 2020-03-15 Suffolk 3.18298578966 2020-03-19 New York City 152.08392429327 2020-03-19 Dutchess 10.53640497862 2020-03-18 Dutchess 6.79768063137 2020-03-12 Saratoga 1.30512522677 2020-03-15 Dutchess 2.37918822098 2020-03-18 Rockland 9.20841403485 2020-03-13 Albany 0.65465162714 2020-03-19 Rockland 16.26819812824 2020-03-13 Orange 0.77934223515 2020-03-19 Monroe 3.77475497796 2020-03-19 Westchester 82.37675011835 2020-03-17 Albany 7.52849371207 2020-03-20 Nassau 55.56685562345 2020-03-15 Westchester 20.15491376798 2020-03-17 Suffolk 5.68874056025 2020-03-13 New York City 9.51675747495 2020-03-16 Nassau 8.03287435405 2020-03-12 Monroe 0.13481267778 2020-03-18 Westchester 55.50353176104 2020-03-18 Orange 8.31298384164 2020-03-13 Rockland 2.76252421046 2020-03-11 Nassau 2.06349065976 2020-03-16 Dutchess 3.39884031568 2020-03-12 Westchester 15.19370422509 2020-03-14 Saratoga 1.30512522677 2020-03-10 New York City 2.27174210692 2020-03-19 Suffolk 12.05471213957 2020-03-16 Monroe 1.34812677784 2020-03-10 Westchester 11.05936293935 2020-03-11 Rockland 1.84168280697 2020-03-09 Saratoga 0.87008348451 2020-03-19 Orange 13.24881799761 2020-03-11 Westchester 12.40302385722 2020-03-20 Albany 19.96687462767 2020-03-09 Rockland 1.22778853798 2020-03-15 Rockland 3.99031274844 2020-03-16 Westchester 22.63551853942 2020-03-09 Westchester 10.02577761792 2020-03-11 Monroe 0.13481267778 2020-03-17 Saratoga 3.91537568030 2020-03-17 Rockland 6.75283695889 2020-03-13 Westchester 16.22728954652 2020-03-12 Rockland 2.14862994147 2020-03-17 New York City 39.66338921819 2020-03-20 Suffolk 25.12527080775 2020-03-14 Monroe 0.26962535557 2020-03-12 Dutchess 0.33988403157 2020-03-18 Nassau 13.48638538341 2020-03-06 Rockland 0.61389426899 2020-03-14 New York City 16.51617910169 2020-03-14 Rockland 3.68336561394 2020-03-12 New York City 5.89424979094 2020-03-20 Rockland 31.00166058400 2020-03-13 Suffolk 1.96396995532 2020-03-14 Nassau 5.82199150431 2020-03-07 New York City 0.73678122387 2020-03-10 Saratoga 0.87008348451 2020-03-10 Rockland 1.84168280697 2020-03-13 Nassau 3.75850084456 2020-03-15 Monroe 0.26962535557 2020-03-16 Orange 2.85758819556 2020-03-15 New York City 20.26148365635 2020-03-14 Suffolk 2.77664717822 2020-03-17 Dutchess 5.43814450509 2020-03-13 Dutchess 1.0196520

## Visualize Cases by State Choropleth Maps
* Join the data with `map_fips_dedup` to obtain the county centroid lat, long_

In [0]:
# Extract Day Number and county centroid lat, long_
wa_daynum = spark.sql("""select f.fips, f.county, f.date, f.day_num, cases as confirmed, cast(f.cases_per_100Kpop as int) as confirmed_per100K, deaths, cast(f.deaths_per_100Kpop as int) as deaths_per100K, m.lat, m.long_ from wa_state_window f join map_fips_dedup m on m.fips = f.fips""")
wa_daynum.createOrReplaceTempView("wa_daynum")
ny_daynum = spark.sql("""select cast(f.fips as int) as fips, f.county, f.date, f.day_num, cases as confirmed, cast(f.cases_per_100Kpop as int) as confirmed_per100K, deaths, cast(f.deaths_per_100Kpop as int) as deaths_per100K, m.lat, m.long_ from ny_state_window f join map_fips_dedup m on m.fips = f.fips""")
ny_daynum.createOrReplaceTempView("ny_daynum")
ny_daynum_m1 = spark.sql("""select cast(f.fips as int) as fips, f.county, f.date, f.day_num, cases as confirmed, cast(f.cases_per_100Kpop as int) as confirmed_per100K, deaths, cast(f.deaths_per_100Kpop as int) as deaths_per100K, m.lat, m.long_ from ny_state_window_m1 f join map_fips_dedup m on m.fips = f.fips""")
ny_daynum_m1.createOrReplaceTempView("ny_daynum_m1")

In [0]:
# Obtain Topography
topo_usa = 'https://vega.github.io/vega-datasets/data/us-10m.json'
topo_wa = 'https://raw.githubusercontent.com/deldersveld/topojson/master/countries/us-states/WA-53-washington-counties.json'
topo_ny = 'https://raw.githubusercontent.com/deldersveld/topojson/master/countries/us-states/NY-36-new-york-counties.json'
us_counties = alt.topo_feature(topo_usa, 'counties')
wa_counties = alt.topo_feature(topo_wa, 'cb_2015_washington_county_20m')
ny_counties = alt.topo_feature(topo_ny, 'cb_2015_new_york_county_20m')

In [0]:
# Review WA
confirmed_wa = wa_daynum.select("fips", "day_num", "date", "confirmed", "confirmed_per100K", "county").where("confirmed > 0").toPandas()
confirmed_wa['date'] = confirmed_wa['date'].astype(str)
deaths_wa = wa_daynum.select("lat", "long_", "day_num", "date", "deaths", "deaths_per100K", "county").where("deaths > 0").toPandas()
deaths_wa['date'] = deaths_wa['date'].astype(str)

# Review NY
confirmed_ny = ny_daynum.select("fips", "day_num", "date", "confirmed", "confirmed_per100K", "county").where("confirmed > 0").toPandas()
confirmed_ny['date'] = confirmed_ny['date'].astype(str)
deaths_ny = ny_daynum.select("lat", "long_", "day_num", "date", "deaths", "deaths_per100K", "county").where("deaths > 0").toPandas()
deaths_ny['date'] = deaths_ny['date'].astype(str)

# Review NY -1 week
confirmed_ny_m1 = ny_daynum_m1.select("fips", "day_num", "date", "confirmed", "confirmed_per100K", "county").where("confirmed > 0").toPandas()
confirmed_ny_m1['date'] = confirmed_ny_m1['date'].astype(str)
deaths_ny_m1 = ny_daynum_m1.select("lat", "long_", "day_num", "date", "deaths", "deaths_per100K", "county").where("deaths > 0").toPandas()
deaths_ny_m1['date'] = deaths_ny_m1['date'].astype(str)

In [0]:
# State Choropleth Map Visualization Function
def map_state(curr_day_num, state_txt, state_counties, confirmed, confirmed_min, confirmed_max, deaths, deaths_min, deaths_max):
  # Get date_str
  date_str = confirmed[confirmed['day_num'] == 101]['date'].head(1).item()
  
  # State
  base_state = alt.Chart(state_counties).mark_geoshape(
      fill='white',
      stroke='lightgray',
  ).properties(
      width=800,
      height=600,
  ).project(
      type='mercator'
  )

  # counties
  base_state_counties = alt.Chart(us_counties).mark_geoshape(
  ).transform_lookup(
    lookup='id',
    from_=alt.LookupData(confirmed[(confirmed['confirmed_per100K'] > 0) & (confirmed['day_num'] == curr_day_num)], 'fips', ['confirmed_per100K', 'confirmed', 'county', 'date', 'fips'])  
  ).encode(
     color=alt.Color('confirmed_per100K:Q', scale=alt.Scale(type='log', domain=[confirmed_min, confirmed_max]), title='Confirmed per 100K'),
    tooltip=[
      alt.Tooltip('fips:O'),
      alt.Tooltip('confirmed:Q'),
      alt.Tooltip('confirmed_per100K:Q'),
      alt.Tooltip('county:N'),
      alt.Tooltip('date:N'),
    ],
  )

  # deaths by long, latitude
  points = alt.Chart(deaths[(deaths['deaths_per100K'] > 0) & (deaths['day_num'] == curr_day_num)]).mark_point(opacity=0.75, filled=True).encode(
    longitude='long_:Q',
    latitude='lat:Q',
    size=alt.Size('sum(deaths_per100K):Q', scale=alt.Scale(type='symlog', domain=[deaths_min, deaths_max]), title='Deaths per 100K'),
    color=alt.value('#BD595D'),
    stroke=alt.value('brown'),
    tooltip=[
      alt.Tooltip('lat'),
      alt.Tooltip('long_'),
      alt.Tooltip('deaths'),
      alt.Tooltip('county:N'),      
      alt.Tooltip('date:N'),      
    ],
  ).properties(
    # update figure title
    title=f'COVID-19 {state_txt} Confirmed Cases and Deaths per 100K by County [{curr_day_num}, {date_str}]'
  )

  return (base_state + base_state_counties + points)


| Factors | WA | NY | 
| ------- | -- | -- | 
| Educational Facilities Closed | 3/13/2020 | 3/18/2020 |
| Day 00 | 3/6/2020 | 3/11/2020 |
| Day 14 | 3/20/2020 | 3/25/2020 | 
| Max Cases | 794 | 20011 |
| Max Deaths | 68 | 280 |
| Max Cases per 100K | 50.55 | 1222.97 | 
| Max Deaths per 100K | 3.27 | 17.11 |

### WA State (cases and deaths per 100K)

In [0]:
map_state(101, 'WA', wa_counties, confirmed_wa, 1, 60, deaths_wa, 1, 5)

alt.LayerChart(...)

In [0]:
map_state(107, 'WA', wa_counties, confirmed_wa, 1, 60, deaths_wa, 1, 5)

alt.LayerChart(...)

In [0]:
map_state(114, 'WA', wa_counties, confirmed_wa, 1, 60, deaths_wa, 1, 5)

alt.LayerChart(...)

### NY State (cases and deaths per 100K)

In [0]:
map_state(101, 'NY', ny_counties, confirmed_ny, 1, 1500, deaths_ny, 1, 20)

alt.LayerChart(...)

In [0]:
map_state(107, 'NY', ny_counties, confirmed_ny, 1, 1500, deaths_ny, 1, 20)

alt.LayerChart(...)

In [0]:
map_state(114, 'NY', ny_counties, confirmed_ny, 1, 1500, deaths_ny, 1, 20)

alt.LayerChart(...)

## COVID-19 Confirmed Cases and Deaths by WA and NY County Slider

In [0]:
# State Choropleth Map Visualization Function
def map_state_slider(state_txt, state_counties, confirmed, confirmed_min, confirmed_max, deaths, deaths_min, deaths_max, state_fips):
  # Pivot confirmed data by day_num
  confirmed_pv = confirmed[['fips', 'day_num', 'confirmed']].copy()
  confirmed_pv['fips'] = confirmed_pv['fips'].astype(str)
  confirmed_pv['day_num'] = confirmed_pv['day_num'].astype(str)
  confirmed_pv['confirmed'] = confirmed_pv['confirmed'].astype('int64')
  confirmed_pv = confirmed_pv.pivot_table(index='fips', columns='day_num', values='confirmed', fill_value=0).reset_index()

  # Pivot deaths data by day_num
  deaths_pv = deaths[['lat', 'long_', 'day_num', 'deaths']].copy()
  deaths_pv['day_num'] = deaths_pv['day_num'].astype(str)
  deaths_pv['deaths'] = deaths_pv['deaths'].astype('int64')
  deaths_pv = deaths_pv.pivot_table(index=['lat', 'long_'], columns='day_num', values='deaths', fill_value=0).reset_index()

  # Extract column names for slider
  column_names = confirmed_pv.columns.tolist()

  # Remove first element (`fips`)
  column_names.pop(0)

  # Convert to int
  column_values = [None] * len(column_names)
  for i in range(0, len(column_names)): column_values[i] = int(column_names[i]) 
  
  # Disable max_rows to see more data
  alt.data_transformers.disable_max_rows()

  # Topographic information
  us_states = alt.topo_feature(topo_usa, 'states')
  us_counties = alt.topo_feature(topo_usa, 'counties')

  # state county boundaries
  base_state = alt.Chart(state_counties).mark_geoshape(
      fill='white',
      stroke='lightgray',
  ).properties(
      width=800,
      height=600,
  ).project(
      type='mercator'
  )

  # Slider choices
  min_day_num = column_values[0]
  max_day_num = column_values[len(column_values)-1]
  slider = alt.binding_range(min=min_day_num, max=max_day_num, step=1)
  slider_selection = alt.selection_single(fields=['day_num'], bind=slider, name="day_num", init={'day_num':min_day_num})


  # Confirmed cases by county
  base_state_counties = alt.Chart(us_counties).mark_geoshape(
      stroke='black',
      strokeWidth=0.05
  ).transform_lookup(
      lookup='id',
      from_=alt.LookupData(confirmed_pv, 'fips', column_names)  
  ).transform_fold(
      column_names, as_=['day_num', 'confirmed']
  ).transform_calculate(
      state_id = "(datum.id / 1000)|0",
      day_num = 'parseInt(datum.day_num)',
      confirmed = 'isValid(datum.confirmed) ? datum.confirmed : -1'
  ).encode(
      color = alt.condition(
          'datum.confirmed > 0',      
          alt.Color('confirmed:Q', scale=alt.Scale(domain=(confirmed_min, confirmed_max), type='symlog')),
          alt.value('white')
        )  
  ).properties(
    # update figure title
    title=f'COVID-19 WA State Confirmed Cases by County'
  ).transform_filter(
      (alt.datum.state_id)==state_fips
  ).transform_filter(
      slider_selection
  )

  # deaths by long, latitude
  points = alt.Chart(
    deaths_pv
  ).mark_point(
    opacity=0.75, filled=True
  ).transform_fold(
    column_names, as_=['day_num', 'deaths']
  ).transform_calculate(
      day_num = 'parseInt(datum.day_num)',
      deaths = 'isValid(datum.deaths) ? datum.deaths : -1'  
  ).encode(
    longitude='long_:Q',
    latitude='lat:Q',
    size=alt.Size('deaths:Q', scale=alt.Scale(domain=(deaths_min, deaths_max), type='symlog'), title='deaths'),
    color=alt.value('#BD595D'),
    stroke=alt.value('brown'),
  ).add_selection(
      slider_selection
  ).transform_filter(
      slider_selection
  )

  # confirmed cases (base_counties) and deaths (points)
  return (base_state + base_state_counties + points) 

## COVID-19 Confirmed Cases and Deaths by WA and NY County Map and Graph

In [0]:
# map_state_graph
def map_state_graph(state_txt, state_counties, confirmed, confirmed_min, confirmed_max, deaths, deaths_min, deaths_max, state_fips):
  
  # pivot confirmed cases (by date)
  confirmed_pv2 = confirmed[['fips', 'date', 'confirmed']].copy()
  confirmed_pv2['fips'] = confirmed_pv2['fips'].astype(str)
  confirmed_pv2['date'] = confirmed_pv2['date'].astype(str)
  confirmed_pv2['confirmed'] = confirmed_pv2['confirmed'].astype('int64')
  confirmed_pv2 = confirmed_pv2.pivot_table(index='fips', columns='date', values='confirmed', fill_value=0).reset_index()

  # pivot deaths
  deaths_pv2 = deaths[['lat', 'long_', 'date', 'deaths']].copy()
  deaths_pv2['date'] = deaths_pv2['date'].astype(str)
  deaths_pv2['deaths'] = deaths_pv2['deaths'].astype('int64')
  deaths_pv2 = deaths_pv2.pivot_table(index=['lat', 'long_'], columns='date', values='deaths', fill_value=0).reset_index()

  # Extract column names for slider
  column_names2 = confirmed_pv2.columns.tolist()

  # Remove first element (`fips`)
  column_names2.pop(0)

  # date selection
  pts = alt.selection(type="single", encodings=['x'])

  # State
  base_state = alt.Chart(state_counties).mark_geoshape(
      fill='white',
      stroke='lightgray',
  ).properties(
      width=800,
      height=600,
  ).project(
      type='mercator'
  )

  # State Counties
  base_state_counties = alt.Chart(us_counties).mark_geoshape(
    stroke='black',
    strokeWidth=0.05,
  ).transform_lookup(
    lookup='id',
   from_=alt.LookupData(confirmed_pv2, 'fips', column_names2)
   ).transform_fold(
     column_names2, as_=['date', 'confirmed']
  ).transform_calculate(
      state_id = "(datum.id / 1000)|0",
      date = 'datum.date',
      confirmed = 'isValid(datum.confirmed) ? datum.confirmed : -1'
  ).encode(
       color = alt.condition(
          'datum.confirmed > 0',      
          alt.Color('confirmed:Q', scale=alt.Scale(domain=(confirmed_min, confirmed_max), type='symlog')),
          alt.value('white')
        )  
  ).transform_filter(
    pts
  ).transform_filter(
      (alt.datum.state_id)==state_fips
  )

  # Bar Graph
  bar = alt.Chart(confirmed).mark_bar().encode(
      x='date:N',
      y='confirmed_per100K:Q',
      color=alt.condition(pts, alt.ColorValue("steelblue"), alt.ColorValue("grey"))
  ).properties(
      width=800,
      height=200,
      title='Confirmed Cases per 100K'
  ).add_selection(pts)

  # Deaths
  points = alt.Chart(deaths).mark_point(opacity=0.75, filled=True).encode(
    longitude='long_:Q',
    latitude='lat:Q',
    size=alt.Size('sum(deaths):Q', scale=alt.Scale(domain=[deaths_min, deaths_max]), title='Deaths'),
    color=alt.value('#BD595D'),
    stroke=alt.value('brown'),
    tooltip=[
      alt.Tooltip('lat'),
      alt.Tooltip('long_'),
      alt.Tooltip('deaths'),
      alt.Tooltip('county:N'),      
      alt.Tooltip('date:N'),      
    ],
  ).properties(
    # update figure title
    title=f'COVID-19 Confirmed Cases and Deaths by County'
  ).transform_filter(
      pts
  )

  return (base_state + base_state_counties + points) & bar

In [0]:
map_state_graph('WA', wa_counties, confirmed_wa, 1, 800, deaths_wa, 1, 70, 53)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-8ba5ff29-f8fb-43f2-8173-6a772610d27e/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection' is deprecated.
   Use 'selection_point()' or 'selection_interval()' instead; these functions also include more helpful docstrings.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/local_disk0/.ephemeral_nfs/envs/pythonEnv-8ba5ff29-f8fb-43f2-8173-6a772610d27e/lib/python3.11/site-packages/altair/vegalite/v5/api.py:469: AltairDeprecationWarning: The types 'single' and 'multi' are now
        combined and should be specified using "selection_point()".
  warnings.warn(
/local_disk0/.ephemeral_nfs/envs/pythonEnv-8ba5ff29-f8fb-43f2-8173-6a772610d27e/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.VConcatChart(...)

In [0]:
map_state_graph('NY', ny_counties, confirmed_ny, 1, 21000, deaths_ny, 1, 300, 36)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-8ba5ff29-f8fb-43f2-8173-6a772610d27e/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection' is deprecated.
   Use 'selection_point()' or 'selection_interval()' instead; these functions also include more helpful docstrings.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/local_disk0/.ephemeral_nfs/envs/pythonEnv-8ba5ff29-f8fb-43f2-8173-6a772610d27e/lib/python3.11/site-packages/altair/vegalite/v5/api.py:469: AltairDeprecationWarning: The types 'single' and 'multi' are now
        combined and should be specified using "selection_point()".
  warnings.warn(
/local_disk0/.ephemeral_nfs/envs/pythonEnv-8ba5ff29-f8fb-43f2-8173-6a772610d27e/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.VConcatChart(...)

In [0]:
map_state_graph('NY', ny_counties, confirmed_ny_m1, 1, 4500, deaths_ny, 1, 70, 36)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-8ba5ff29-f8fb-43f2-8173-6a772610d27e/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection' is deprecated.
   Use 'selection_point()' or 'selection_interval()' instead; these functions also include more helpful docstrings.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/local_disk0/.ephemeral_nfs/envs/pythonEnv-8ba5ff29-f8fb-43f2-8173-6a772610d27e/lib/python3.11/site-packages/altair/vegalite/v5/api.py:469: AltairDeprecationWarning: The types 'single' and 'multi' are now
        combined and should be specified using "selection_point()".
  warnings.warn(
/local_disk0/.ephemeral_nfs/envs/pythonEnv-8ba5ff29-f8fb-43f2-8173-6a772610d27e/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.VConcatChart(...)